# WikidWiki Class Description

## Implemented Attributes
- rootfolder
- datafolder
- pageNames[]: list of all pages
- pageNameSet
- pageCount
    
## Implemented Methods

#### __init__(self,folder_path):
Construct a WikidWiki class using the folder_path as string.

#### readFileSystem(self):
Populates the PageName attributes of WikidWiki class from the filesystem


### Single Page Functions:
#### doesPageContainStringLowMem(self,pageName,srchstr):
#### doesPageContainString(self,pageName,srchstr):
#### __extractWikiFileNamesFromPath(self,wikiFilePath):
#### __getFilePathFromPageName(self,pageName):


### Get Pages:
#### getAllPages(self):
Returns a list of Pages found in data folder.

#### getPagesByFunction(self,matchFunction,matchValue=""):
Passed a match function and a match value to filter pages
Returns list of pageNames that return true when processed by function matchFunction(matchValue,pagename)

#### getPagesBySearchStr(self,srchstr):
Returns list of pageNames that contain the srchstr in their content

#### getPagesBySearchStrLowMem(self,srchstr):
Returns list of pageNames that contain the srchstr in their content

#### getPagesSetUnion(self,pn1,pn2):
Returns the union of pageName lists pn1 and pn2

#### getPagesSetIntersection(self,pn1,pn2):
Returns the intersection of pageName lists pn1 and pn2

#### getPagesSetDifference(self,pn1,pn2):
Returns the difference of pageName lists pn1 and pn2

#### getPagesSetSymmetricDifference(self,pn1,pn2):
Returns the symmetric difference (those in one or the other, but not both) of pageName lists pn1 and pn2



In [8]:
import os,glob
import mmap

from WikidWorker import hasWiki

class WikidWiki(object):
    """class for working with a wikidpad wiki from the filesystem."""

    def __init__(self,folder_path):
        """Construct a WikidWiki class using the folder_path as string."""
        # raise error if folder path doesn't exist or contains no wiki:
        if not os.path.exists(folder_path):
            raise Exception("folder " + folder_path + " does not exist!")
        if not hasWiki(folder_path):
            raise Exception("folder " + folder_path + " does not contain a wiki!")

        #set attributes:
        self.rootfolder = folder_path
        self.datafolder = os.path.join(folder_path,'data')
        self.readFileSystem()
        
    def readFileSystem(self):
        """Populates the PageName attributes of WikidWiki class from the filesystem"""
        self.pageNames = self.getAllPages()
        self.pageNamesSet = set(self.pageNames)
        self.pageCount = len(self.pageNames)
    
    def __extractWikiFileNamesFromPath(self,wikiFilePath):
        filename = os.path.split(wikiFilePath)[1]
        return os.path.splitext(filename)[0]

    def __getFilePathFromPageName(self,pageName):
        filename = os.path.join(self.datafolder,"%s.wiki" % pageName)
        return filename    
    
    def getAllPages(self):
        """Returns a list of Pages found in data folder."""
        filePathList = glob.glob(self.datafolder +'/*.wiki')
        return [self.__extractWikiFileNamesFromPath(x) for x in filePathList]
    
    def getPagesByFunction(self,matchFunction,matchValue=""):
        """ Passed a match function and a match value to filter pages
            Returns list of pageNames that return true when processed by function matchFunction(matchValue,pagename)
        """
        self.pageNames = self.getAllPages()
        if matchValue:
            return [x for x in self.pageNames if matchFunction(matchValue,x)]
        else:
            return [x for x in self.pageNames if matchFunction(x)]            

    def doesPageContainStringLowMem(self,pageName,srchstr):
        filepath = self.__getFilePathFromPageName(pageName)
        
        if os.stat(filepath).st_size > 0:
            f = open(filepath)
            s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
            return s.find(srchstr) != -1
        return False
    
    def doesPageContainString(self,pageName,srchstr):
        filepath = self.__getFilePathFromPageName(pageName)
        return srchstr in open(filepath).read()

    def getPagesBySearchStr(self,srchstr):
        """Returns list of pageNames that contain the srchstr in their content"""
        return [x for x in self.pageNames if self.doesPageContainString(x,srchstr)]  
    
    def getPagesBySearchStrLowMem(self,srchstr):
        """Returns list of pageNames that contain the srchstr in their content"""
        return [x for x in self.pageNames if self.doesPageContainStringLowMem(x,srchstr)] 
    
    def getPagesSetUnion(self,pn1,pn2):
        """Returns the union of pageName lists pn1 and pn2"""
        ps1 = set(pn1)
        ps2 = set(pn2)
        return ps1 | ps2
    
    def getPagesSetIntersection(self,pn1,pn2):
        """Returns the intersection of pageName lists pn1 and pn2"""
        ps1 = set(pn1)
        ps2 = set(pn2)
        return ps1 & ps2

    def getPagesSetDifference(self,pn1,pn2):
        """Returns the difference of pageName lists pn1 and pn2"""
        ps1 = set(pn1)
        ps2 = set(pn2)
        return ps1 - ps2

    def getPagesSetSymmetricDifference(self,pn1,pn2):
        """Returns the symmetric difference (those in one or the other, but not both) of pageName lists pn1 and pn2"""
        ps1 = set(pn1)
        ps2 = set(pn2)
        return ps1 ^ ps2

In [9]:
## MANUAL TESTS

# w = WikidWiki('TestWiki')
# w = WikidWiki('BadTestWiki')
# w.getAllWikiPages()

#JWiki test:

# wik = WikidWiki("/Users/jaysen/DEV/Projects/WikidLib/NewtWiki")
# print wik.pageCount

jwik = WikidWiki("/Users/jaysen/DEV/Projects/WikidLib/JWiki")
print "pagecount = %i" % jwik.pageCount
print
s1 = "[tag:music]"
s2 = "[tag:southafrica]"
pn1 = jwik.getPagesBySearchStr(s1)
pn2 = jwik.getPagesBySearchStr(s2)

print "Some searches:"
print "pn1 searched by '%s'. count = %i" % (s1,len(pn1)) 
print pn1
print
print "pn2 searched by '%s'. count = %i" % (s2,len(pn2))
print pn2
print

ps1 = jwik.getPagesSetUnion(pn1,pn2)
print "ps1 (Union(pn1,pn2). count = %i" % len(ps1)
print ps1
print
ps2 = jwik.getPagesSetIntersection(pn1,pn2)
print "ps2 (Intersection(pn1,pn2). count = %i" % len(ps2)
print ps2
print
ps3 = jwik.getPagesSetDifference(pn1,pn2)
print "ps3 (Difference(pn1,pn2). count = %i" % len(ps3)
print ps3
print
ps4 = jwik.getPagesSetSymmetricDifference(pn1,pn2)
print "ps4 (SymmetricDifference(pn1,pn2). count = %i" % len(ps4)
print ps4

pagecount = 2266

Some searches:
pn1 searched by '[tag:music]'. count = 12
['FavMusic', 'GrahamstownFestival', 'JoyOfJazz', 'KDf', 'LastFm', 'LibreFm', 'ManOfLaMancha', 'MusicWiki', 'PyMp3', 'ShivavA', 'SongLyrics', 'TodoMUSIC']

pn2 searched by '[tag:southafrica]'. count = 40
['AhmedVeriava', 'AndileMngxitama', 'AshwinDesai', 'BraaiCriminalsHoliday', 'CullinanFarm', 'DhireshRamklass', 'EightTa3G', 'ExclusiveBooks', 'GhandiSquare', 'GroblersdalTown', 'JhB', 'JoyOfJazz', 'KDf', 'LeeWalters', 'MarlothPark', 'MWeb', 'PayFine', 'PlaceSophiaTown', 'PrePaidWaterMeter', 'PrincE', 'RajeshRamchunder', 'RentAWreck', 'RiverdeneHouseTelkom', 'SaBC', 'SaRS', 'ShingalanaLodge', 'ShivavA', 'SizweShabalala', 'SocPen', 'SophiaTownClub', 'SouthAfricanPolitical', 'StdBank', 'SteveBiko', 'TeeKay', 'TelKom', 'TheNewtown', 'UbuntuLocoTeams', 'VishalSewpersad', 'VodaCom3g', 'XarraBooks']

ps1 (Union(pn1,pn2). count = 49
set(['GroblersdalTown', 'ShivavA', 'RentAWreck', 'EightTa3G', 'PayFine', 'KDf', 'MusicWik

# Automated Tests:

In [10]:
#TESTS
from nose.tools import assert_equal
from nose.tools import assert_items_equal



class WikidWikiClassTest(object):
    
    def test(self):      
        tc = WikidWiki('TestWiki')
        
        assert_equal(tc.pageNames,['TestOne', 'TestThree', 'TestTwo'])
        assert_equal(tc.pageCount,3)
        
        assert_items_equal(tc.getPagesByFunction(PageNameContainsStringOne),['TestOne'])
        assert_items_equal(tc.getPagesByFunction(PageNameContainsString,'Two'),['TestTwo'])
        assert_items_equal(tc.getPagesByFunction(PageNameContainsString,'Three'),['TestThree'])
        assert_items_equal(tc.getPagesByFunction(PageNameContainsString,'Test'),['TestOne','TestTwo','TestThree'])
        
        print 'class ' + str(tc) + ' - PASSED ALL TESTS'
                     
#run tests
def PageNameContainsStringOne(pageName):
    return 'One' in pageName

def PageNameContainsString(s,pageName):
    return s in pageName

t = WikidWikiClassTest()
t.test()


class <__main__.WikidWiki object at 0x1042ace90> - PASSED ALL TESTS


## PageNameSet Class

In [94]:
class PageNameSet (object):
    
    def __init__(self,pageName):
        
        self.filePath
        
        
        

# Methods todo

#### getPagesByTags( tag | tagList)
